# Prepare the Environment

!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!nvcc --version

! apt-get install gcc-7 g++-7
!which gcc-7
! update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-7 70 --slave /usr/bin/g++ g++ /usr/bin/g++-7
!gcc --version

# START HERE

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-oku4mr80
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-oku4mr80
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=a56c3e7f6664e8a60d21e0c58acbc628a4ff8fe00d39511061aa69ef58c0dacc
  Stored in directory: /tmp/pip-ephem-wheel-cache-rob7g0oo/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [24]:
%cd /content/drive/MyDrive/CUDA
!chmod +x Merge.sbatch

/content/drive/MyDrive/CUDA


In [3]:
!pwd

/content/drive/MyDrive/CUDA


In [65]:
!./Merge.sbatch

Merge_Sort By Sami Osman
Array size:          1000
Block size:          10
Analysis flag        false
Shared memory flag:  false
Saving unsorted array to Unsorted.txt file ...
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@ MergeSortWithCuda @@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
First loop Configuration
Block/Grid: 10 
Thread/Block: 51 
Total threads: 510 
T_Time: 0.000652
@@@@@@@@@@@@@@@@@@@@@@@@
@@ SORTING IS CORRECT @@
@@@@@@@@@@@@@@@@@@@@@@@@
Saving sorted array to sorted.txt file ...


In [5]:
!nvidia-smi

Fri Feb  3 08:24:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    32W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-01d03175-82e7-bc7a-f495-50df205c3e84)


In [14]:
%%cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <stdio.h>
    int main() {
        int num = 8;
        switch (num) {
            case 7:
                printf("Value is 7");
                break;
            case 8:
                printf("Value is 8");
                break;
            case 9:
                printf("Value is 9");
                break;
            default:
                printf("Out of range");
                break;
        }
        return 0;
    }


Value is 8


In [8]:
%%cu
#include <iostream>

int main(){
    int x = 2560000 * sizeof(int);
    printf ("%d", x); 
}


10240000


In [9]:
%%cu
#include <iostream>
	int
	main()
{
	std::cout << "Welcome To GeeksforGeeks\n";
	return 0;
}


Welcome To GeeksforGeeks



In [10]:
%%cu

#include <iostream>
__global__ void add(int a, int b, int *c) {
  *c = a + b;
}

int main() {
  int a = 2, b = 3, c;
  int *dev_c;
  cudaMalloc(&dev_c, sizeof(int));
  add<<<1, 1>>>(a, b, dev_c);
  cudaMemcpy(&c, dev_c, sizeof(int), cudaMemcpyDeviceToHost);
  printf("%d + %d = %d\n", a, b, c);
  cudaFree(dev_c);
  return 0;
}


2 + 3 = 5



In [11]:
%%cu
#include <stdio.h>

void merge(int arr[], int l, int m, int r) {
    int i, j, k;
    int n1 = m - l + 1;
    int n2 = r - m;
    int L[n1], R[n2];
    
    for (i = 0; i < n1; i++) {
        L[i] = arr[l + i];
    }
    for (j = 0; j < n2; j++) {
        R[j] = arr[m + 1 + j];
    }
    
    i = 0;
    j = 0;
    k = l;
    
    while (i < n1 && j < n2) {
        if (L[i] <= R[j]) {
            arr[k] = L[i];
            i++;
        } else {
            arr[k] = R[j];
            j++;
        }
        k++;
    }
    
    while (i < n1) {
        arr[k] = L[i];
        i++;
        k++;
    }
    
    while (j < n2) {
        arr[k] = R[j];
        j++;
        k++;
    }
}

void mergeSort(int arr[], int l, int r) {
    if (l < r) {
        int m = l + (r - l) / 2;
        mergeSort(arr, l, m);
        mergeSort(arr, m + 1, r);
        merge(arr, l, m, r);
    }
}

int main() {
    int arr[] = {12, 11, 13, 5, 6, 7};
    int arr_size = sizeof(arr) / sizeof(arr[0]);
    printf("Given array is \n");
    int i;
    for (i = 0; i < arr_size; i++) {
        printf("%d ", arr[i]);
    }
    
    mergeSort(arr, 0, arr_size - 1);
    
    printf("\nSorted array is \n");
    for (i = 0; i < arr_size; i++) {
        printf("%d ", arr[i]);
    }
    return 0;
}


Given array is 
12 11 13 5 6 7 
Sorted array is 
5 6 7 11 12 13 
